In [ ]:
import pandas as pd
import twint
import nest_asyncio
from datetime import date

def tweet_scrape(search_term):
    nest_asyncio.apply()
    c = twint.Config()
    c.Lang = "en"
    c.Since = str(date.today())
    c.Search = [search_term]
    c.Pandas = True
    twint.run.Search(c)
    Tweets_df = twint.storage.panda.Tweets_df
    return Tweets_df


Tweets_df = tweet_scrape('hy/droflask')


In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def vader_scores(tweet):
    """
    Run VADER sentiment analysis and return  values
    """
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    return list(vs.values())


def score_columns(Tweets_df):
    """
    Apply those values to the tweets_df and then seperate them into distinct columns
    """
    Tweets_df['Sentiment'] = Tweets_df['tweet'].map(vader_scores)
    Tweets_df[['negative','neutral','positive', 'compound']] = Tweets_df['Sentiment'].tolist()
    Tweets_df.drop(columns='Sentiment', inplace=True)




In [20]:
from google.cloud import bigquery


def load_bq(dataframe, table):
    client = bigquery.Client()
    df = dataframe
    job = client.load_table_from_dataframe(df, table)
    job.result()


load_bq(Tweets_df,'deb-01-346001.vibe_shift.hydroflask')

https://datastudio.google.com/s/vAweiTS-KgE

In [ ]:
import pandas as pd
import twint
import nest_asyncio
from datetime import date

def tweet_scrape():
    nest_asyncio.apply()
    c = twint.Config()
    c.Lang = "en"
    c.Since = '2022-07-21'
    c.Username = "Grimezsz"
    c.Pandas = True
    twint.run.Search(c)
    Tweets_df = twint.storage.panda.Tweets_df
    return grimes_df


grimes_df = tweet_scrape()

In [4]:
from config import database, table, search_term
import pandas as pd
import twint
import nest_asyncio
from datetime import date

def tweet_scrape(search_term):
    nest_asyncio.apply()
    c = twint.Config()
    c.Lang = "en"
    c.Since = str(date.today())
    c.Search = [search_term]
    c.Pandas = True
    twint.run.Search(c)
    Tweets_df = twint.storage.panda.Tweets_df
    return Tweets_df

heat_df = tweet_scrape(search_term)

1551613307374764032 2022-07-25 10:00:14 -0700 <DKReignmakers> The second Reignmakers UFC drop is coming out swinging just in time for UFC 277. Get the newest Heatwave drop TOMORROW and you could win a pair of autographed gloves!  Learn more:  https://t.co/S6yW2bDOQ5  https://t.co/yIlx3S4jWu
1551613303998259200 2022-07-25 10:00:13 -0700 <rockblockradio> Now playing: Heat Waves (Oliver Heldens Extended Mix) by @GlassAnimals #listen at  https://t.co/iEDm4lJ84J  https://t.co/29isH73g3T
1551613301511135233 2022-07-25 10:00:13 -0700 <TheLastWord> Record temperatures forecast as heat wave blasts across U.S.  https://t.co/tY9y0NqUxR  https://t.co/qtaA278YgK
1551613296012558336 2022-07-25 10:00:12 -0700 <betincrypto> Crypto miners in Texas shut down operations as state experiences extreme heat wave  Toplist Crypto Casino &amp; SportsBook 👉  https://t.co/4RPbxhUeNy 👈  https://t.co/4bNoXMMMHW
1551613292837232642 2022-07-25 10:00:11 -0700 <OldDrow> London narrowly avoided post heatwave blackout. I

In [32]:
from google.cloud import bigquery


def load_bq(dataframe, table):
    client = bigquery.Client()
    df = dataframe
    job_config = bigquery.LoadJobConfig(
        schema = [
            {'name': 'id', 'type': 'STRING'},
            {'name': 'conversation_id', 'type': 'STRING'},
            {'name': 'created_at', 'type': 'STRING'},
            {'name': 'date', 'type': 'DATETIME'},
            {'name': 'timezone', 'type': 'STRING'},
            {'name': 'place', 'type': 'STRING'},
            {'name': 'tweet', 'type': 'STRING'},
            {'name': 'language', 'type': 'STRING'},
            {'name': 'hashtags', 'type': 'ARRAY'},
            {'name': 'cashtags', 'type': 'ARRAY'},
            {'name': 'user_id', 'type': 'STRING'},
            {'name': 'user_id_str', 'type': 'STRING'},
            {'name': 'username', 'type': 'STRING'},
            {'name': 'name', 'type': 'STRING'},
            {'name': 'day', 'type': 'INT64'},
            {'name': 'hour', 'type': 'INT64'},
            {'name': 'link', 'type': 'STRING'},
            {'name': 'urls', 'type': 'STRING'},
            {'name': 'photos', 'type': 'STRING'},
            {'name': 'video', 'type': 'STRING'},
            {'name': 'thumbnail', 'type': 'STRING'},
            {'name': 'retweet', 'type': 'BOOL'},
            {'name': 'nlikes', 'type': 'INT64'},
            {'name': 'nreplies', 'type': 'INT64'},
            {'name': 'nretweets', 'type': 'INT64'},
            {'name': 'quote_url', 'type': 'STRING'},
            {'name': 'search', 'type': 'STRING'},
            {'name': 'near', 'type': 'STRING'},
            {'name': 'geo', 'type': 'STRING'},
            {'name': 'source', 'type': 'STRING'},
            {'name': 'user_rt_id', 'type': 'STRING'},
            {'name': 'user_rt', 'type': 'STRING'},
            {'name': 'retweet_id', 'type': 'STRING'},
            {'name': 'reply_to', 'type': 'STRING'},
            {'name': 'retweet_date', 'type': 'STRING'},
            {'name': 'translate', 'type': 'STRING'},
            {'name': 'trans_src', 'type': 'STRING'},
            {'name': 'trans_dest', 'type': 'STRING'}
    ])
    job = client.load_table_from_dataframe(df, table, job_config=job_config)
    job.result()

schema = {'id': str, 
            'conversation_id': str, 
            'created_at': str,
            'date': str,
            'timezone': str,
            'place': str,
            'tweet': str,
            'language': str,
            'hashtags': object,
            'cashtags': object,
            'user_id': str,
            'user_id_str': str,
            'username': str,
            'name': str,
            'day': int,
            'hour': int,
            'link': str,
            'urls': str,
            'photos': str,
            'video': str,
            'thumbnail': str,
            'retweet': bool,
            'nlikes': int,
            'nreplies': int,
            'nretweets': int,
            'quote_url': str,
            'search': str,
            'near': str,
            'geo': str,
            'source': str,
            'user_rt_id': str,
            'user_rt': str,
            'retweet_id': str,
            'reply_to': str,
            'retweet_date': str,
            'translate': str,
            'trans_src': str,
            'trans_dest': str}
            
heatsample_df = heat_df.sample(frac=0.25)

heatsample_df = heatsample_df.astype(schema)
load_bq(heatsample_df, f'{database}{table}')

heatsample_df.dtypes

ArrowTypeError: Expected a string or bytes dtype, got float64

In [5]:
import snscrape
import pandas as pd

dune_df = pd.read_json('little_miss_sentiment.json', lines=True)
dune_df = dune_df[['id', 'date', 'content', 'lang', 'url']]


dune_df.head()

,id,date,content,lang,url
0,"{'0': 1550269353194672130, '1': 15502693476749...","{'0': 1658447991000, '1': 1658447989000, '2': ...",{'0': 'his little bye 😭☹️ ive missed him sm gl...,"{'0': 'en', '1': 'en', '2': 'en', '3': 'en', '...",{'0': 'https://twitter.com/wcstsidesugg/status...


In [12]:
#alternate GBQ
    # client = bigquery.Client()
    # sent_df = pd.read_json(f'/opt/airflow/dags/data/{table}_sentiment.json', lines=True)
    # schema = [{'name':'id', 'type': 'STRING'},
    #     {'name':'date', 'type': 'DATETIME'},
    #     {'name': 'content', 'type': 'STRING'},
    #     {'name': 'lang', 'type': 'STRING'},
    #     {'name': 'url', 'type': 'STRING'},
    #     {'name': 'negative', 'type': 'FLOAT'},
    #     {'name': 'neutral', 'type': 'FLOAT'},
    #     {'name': 'positive', 'type': 'FLOAT'},
    #     {'name': 'copmpund', 'type': 'FLOAT'}]
    # sent_df.to_gbq(f'{dataset}.{table}', project_id=f'{project_id}', if_exists='append', table_schema=schema)

# def to_parquet(Tweets_df):
#     Tweets_df.to_parquet(f'/opt/airflow/dags/data/{table}.parquet')

# def load_bq(dataframe, table):
#     client = bigquery.Client()
#     # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/google_creds/goog_creds.json"
#     df = dataframe
#     job = client.load_table_from_dataframe(df, table)
#     job.result()

'2022-07-28'